In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time
import requests
import hmac
import os
import json
from dotenv.main import load_dotenv
from hashlib import sha256

In [9]:
load_dotenv("./dev.env")
api_url="https://open-api.bingx.com"
api_key=os.getenv("api_key")
secret_key=os.getenv("secret_key")

In [17]:
def data_mining(crypto_pair, interval, start_time, end_time):
    payload = {}
    path = '/openApi/swap/v2/quote/klines'
    method = "GET"
    start_timestamp=int(time.mktime(datetime.datetime.strptime(start_time,"%Y/%m/%d").timetuple())*1000)
    end_timestamp=int(time.mktime(datetime.datetime.strptime(end_time,"%Y/%m/%d").timetuple())*1000)
    timestamps=[]
    data=[]
    for i in range(start_timestamp, end_timestamp+1, 432000000):
        timestamps.append(i)
    
    for j in range(len(timestamps)-1):
        paramsMap = {
            "symbol": crypto_pair,
            "interval": interval,
            "startTime": timestamps[j],
            "endTime": timestamps[j+1],
            "limit": 1440
        }
        paramsStr = praseParam(paramsMap)
        data_dict=json.loads(send_request(method, path, paramsStr, payload))['data']
        data_frame=pd.DataFrame(data_dict)
        data.append(data_frame)
    
    return data_preprocessing(data)
    
def get_sign(api_secret, payload):
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    return signature


def send_request(method, path, urlpa, payload):
    url = "%s%s?%s&signature=%s" % (api_url, path, urlpa, get_sign(secret_key, urlpa))
    headers = {
        'X-BX-APIKEY': api_key,
    }
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text

def praseParam(paramsMap):
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])
    return paramsStr+"&timestamp="+str(int(time.time() * 1000))

def milisec_to_sec(x):
    return x/1000

def data_preprocessing(data_list):
    df=pd.concat(data_list, ignore_index=True)
    df['time']=df['time'].apply(milisec_to_sec)
    df['time']=df['time'].apply(datetime.datetime.fromtimestamp)
    df=df.set_index('time')
    return df

In [20]:
df=data_mining("ETH-USDT", "5m", "2023/01/01", "2023/01/31")

In [21]:
df

,open,close,high,low,volume
time,,,,,
2023-01-01 00:00:00,1202.40,1202.55,1202.75,1202.35,423.24
2023-01-01 00:05:00,1202.65,1202.15,1202.65,1201.80,299.21
2023-01-01 00:10:00,1202.15,1202.85,1202.95,1202.00,427.05
2023-01-01 00:15:00,1202.85,1203.00,1203.20,1202.80,879.34
2023-01-01 00:20:00,1203.00,1201.75,1203.00,1201.75,289.37
...,...,...,...,...,...
2023-01-30 23:35:00,1553.85,1550.50,1554.30,1550.35,618.18
2023-01-30 23:40:00,1550.50,1549.90,1551.55,1549.60,433.23
2023-01-30 23:45:00,1549.70,1549.00,1550.40,1547.50,557.44
